In [85]:
import csv
import pandas as pd
import os
from pathlib import Path
import datetime

In [86]:
def iso_to_unix(iso_str):
    # Handles ISO 8601 with 'Z' (UTC)
    # 2025-06-29T08:08:20Z
    try:
        dt = datetime.datetime.strptime(iso_str, "%Y-%m-%dT%H:%M:%SZ")
        dt = dt.replace(tzinfo=datetime.timezone.utc)
        return int(dt.timestamp())
    except ValueError as e:
        print(f"erro: {iso_str}")
        raise e

In [87]:
def parse_csv(file_path):
    csv_file = []
    idx = 0
    with open(file_path, mode='r', newline='', encoding='utf-8') as f:
        reader = csv.reader(f)
        for row in reader:
            if (idx == 0):
                idx+=1
                continue
            row[0] = iso_to_unix(row[0])
            row[1] = float(row[1])
            csv_file.append(row)
    return csv_file     

In [88]:
files = {}
data_dir = Path(Path.cwd().parent / "La Vista 1H")
for file_name in os.listdir(data_dir):
    content = parse_csv(
        data_dir / file_name
    )
    files[file_name] = content



In [89]:
files["Sales Meter Flow Rate (MCF_Day).csv"][1]

[1751184620, 567.95]

## cleaning up logic
cleaning the data before 1st proper start cycle

In [90]:
for file in files:
    print(file)

Sales Meter Static Pressure (PSIA).csv
Line Pressure (PSIA).csv
Casing Pressure (PSI).csv
Tubing Pressure (PSI).csv
Arrival Time Remaining.csv
Sales Meter Flow Rate (MCF_Day).csv
Arrival Speed.csv
Current Non-Arrival Count.csv
Down Hole Pressure (PSI).csv


In [91]:
files["Sales Meter Flow Rate (MCF_Day).csv"]

[[1751184500, 585.2],
 [1751184620, 567.95],
 [1751184740, 0.0],
 [1751184860, 0.0],
 [1751184980, 0.0],
 [1751185100, 1054.75],
 [1751185220, 650.64],
 [1751185340, 613.71],
 [1751185460, 542.38],
 [1751185580, 611.62],
 [1751185700, 573.69],
 [1751185820, 517.6],
 [1751185940, 0.0],
 [1751186060, 0.0],
 [1751186180, 0.0],
 [1751186300, 0.0],
 [1751186420, 617.83],
 [1751186540, 515.95],
 [1751186660, 476.27],
 [1751186780, 411.39],
 [1751186900, 688.06],
 [1751187020, 595.41],
 [1751187140, 580.13],
 [1751187260, 0.0],
 [1751187380, 0.0],
 [1751187500, 0.0],
 [1751187620, 0.0],
 [1751187740, 709.58],
 [1751187860, 587.85],
 [1751187980, 542.48],
 [1751188100, 437.1],
 [1751188220, 604.91],
 [1751188340, 568.88],
 [1751188460, 50.51],
 [1751188580, 0.0],
 [1751188700, 0.0],
 [1751188820, 0.0],
 [1751188940, 732.92],
 [1751189060, 544.09],
 [1751189180, 520.34],
 [1751189300, 471.18],
 [1751189420, 403.82],
 [1751189540, 658.41],
 [1751189660, 584.66],
 [1751189780, 49.72],
 [175118990

In [92]:
files["Sales Meter Flow Rate (MCF_Day).csv"][1][0] + 60

1751184680

In [93]:
def cleanup():
    isotime_threshold = 0
    for i in range(len(files["Sales Meter Flow Rate (MCF_Day).csv"])):
        if files["Sales Meter Flow Rate (MCF_Day).csv"][i][1] == 0.0:
            if i==0: # if list is already clean (starting from zero)
                break
            isotime_threshold = files["Sales Meter Flow Rate (MCF_Day).csv"][i-1][0] + 60 # added one minute of threshold
            files["Sales Meter Flow Rate (MCF_Day).csv"] = files["Sales Meter Flow Rate (MCF_Day).csv"][i:]
            break
    # now cleaning all the data using this in all files
    for file in files:
        if(file=="Sales Meter Flow Rate (MCF_Day).csv"):
            continue
        for i in range(len(files[file])):
            if files[file][i][0] >= isotime_threshold:
                #print(f"threshold: ${isotime_threshold} and data: ${files[file][i][0]}\n")
                files[file] = files[file][i:]  # remove the first element
                break

cleanup()

## Separate Out cycles


In [94]:
def flow_rate_cycles():
    flow_rate = files['Sales Meter Flow Rate (MCF_Day).csv']
    data = [[None, None, None] for i in range(len(flow_rate))]
    cycle_id = -1
    new_cycle_mil_gia = False
    for i in range(len(data)):
        if flow_rate[i][1] == 0.0 and not new_cycle_mil_gia:
            cycle_id+=1
            new_cycle_mil_gia = True
        elif flow_rate[i][1] > 0.0:
            new_cycle_mil_gia = False
        data[i][0] = cycle_id
        data[i][1] = flow_rate[i][0]
        data[i][2] = flow_rate[i][1]

    return data

In [95]:
files['Sales Meter Flow Rate (MCF_Day).csv'] = flow_rate_cycles()
files['Sales Meter Flow Rate (MCF_Day).csv']

[[0, 1751184740, 0.0],
 [0, 1751184860, 0.0],
 [0, 1751184980, 0.0],
 [0, 1751185100, 1054.75],
 [0, 1751185220, 650.64],
 [0, 1751185340, 613.71],
 [0, 1751185460, 542.38],
 [0, 1751185580, 611.62],
 [0, 1751185700, 573.69],
 [0, 1751185820, 517.6],
 [1, 1751185940, 0.0],
 [1, 1751186060, 0.0],
 [1, 1751186180, 0.0],
 [1, 1751186300, 0.0],
 [1, 1751186420, 617.83],
 [1, 1751186540, 515.95],
 [1, 1751186660, 476.27],
 [1, 1751186780, 411.39],
 [1, 1751186900, 688.06],
 [1, 1751187020, 595.41],
 [1, 1751187140, 580.13],
 [2, 1751187260, 0.0],
 [2, 1751187380, 0.0],
 [2, 1751187500, 0.0],
 [2, 1751187620, 0.0],
 [2, 1751187740, 709.58],
 [2, 1751187860, 587.85],
 [2, 1751187980, 542.48],
 [2, 1751188100, 437.1],
 [2, 1751188220, 604.91],
 [2, 1751188340, 568.88],
 [2, 1751188460, 50.51],
 [3, 1751188580, 0.0],
 [3, 1751188700, 0.0],
 [3, 1751188820, 0.0],
 [3, 1751188940, 732.92],
 [3, 1751189060, 544.09],
 [3, 1751189180, 520.34],
 [3, 1751189300, 471.18],
 [3, 1751189420, 403.82],
 [3,

In [96]:
import pandas as pd
df = pd.DataFrame(files['Sales Meter Flow Rate (MCF_Day).csv'], columns=['cycle_id', 'isotime', 'flow_rate'])
df

,cycle_id,isotime,flow_rate
0,0,1751184740,0.00
1,0,1751184860,0.00
2,0,1751184980,0.00
3,0,1751185100,1054.75
4,0,1751185220,650.64
...,...,...,...
7193,680,1752048020,426.05
7194,680,1752048140,614.65
7195,680,1752048260,574.28
7196,680,1752048380,544.98


In [97]:
def data_entries_manager():
    for file_name in files:
        if file_name == "Sales Meter Flow Rate (MCF_Day).csv":
            continue
        df[file_name] = None  # Create a new column for each file
        # Iterate through the DataFrame and for each isotime of flow rate (threshold_isotime)
        # find the corresponding isotime in the range of the thereshold_isotime_range
        # and assign the value from the corresponding file to the DataFrame      
        for i in range(0,7198):
            threshold_isotime = df.iloc[i]["isotime"]
            threshold_isotime_range = [threshold_isotime-60, threshold_isotime+61] # last is not included soo +61 to get +60
            for j in range(len(files[file_name])):
                if files[file_name][j][0] in range(threshold_isotime_range[0], threshold_isotime_range[1]):
                    # found the data entry
                    df.at[i,file_name] = files[file_name][j][1]
                    break
       
data_entries_manager()

In [98]:
df

,cycle_id,isotime,flow_rate,Sales Meter Static Pressure (PSIA).csv,Line Pressure (PSIA).csv,Casing Pressure (PSI).csv,Tubing Pressure (PSI).csv,Arrival Time Remaining.csv,Arrival Speed.csv,Current Non-Arrival Count.csv,Down Hole Pressure (PSI).csv
0,0,1751184740,0.00,97.06,16.52,204.41,181.19,30.0,979.15,0.0,291.14
1,0,1751184860,0.00,96.14,16.49,207.57,196.42,30.0,979.15,0.0,299.94
2,0,1751184980,0.00,95.36,16.55,211.6,202.05,30.0,979.15,0.0,306.28
3,0,1751185100,1054.75,113.09,16.52,215.27,125.86,29.64,979.15,0.0,305.62
4,0,1751185220,650.64,103.22,16.48,215.22,101.29,26.63,979.15,0.0,289.12
...,...,...,...,...,...,...,...,...,...,...,...
7193,680,1752048020,426.05,94.15,16.52,207.56,83.23,22.34,1122.95,0.0,295.53
7194,680,1752048140,614.65,97.66,16.52,205.56,99.8,30.0,1057.8,0.0,291.17
7195,680,1752048260,574.28,97.9,16.49,205.51,98.35,30.0,1057.8,0.0,281.86
7196,680,1752048380,544.98,98.3,16.55,204.19,97.3,30.0,1057.8,0.0,268.87


Events

In [99]:
def generate_events_per_cycle(df, basic_event_funcs, complex_event_funcs):
    events = []
    for _, group in df.groupby('cycle_id'):
        basci_cycle_events = []
        for event_func in basic_event_funcs:
            if callable(event_func):
                basci_cycle_events.append(event_func(group))
        complex_cycle_events = []
        for event_func in complex_event_funcs:
            if callable(event_func):
                complex_cycle_events.append(event_func(group, basci_cycle_events))

        events.extend(basci_cycle_events)
        events.extend(complex_cycle_events)
    return events

basic:

In [100]:
def BasicPressureEvent(cycle_df, SG=0.6, hl=1000):
    cols = [
        "Tubing Pressure (PSI).csv",
        "Casing Pressure (PSI).csv",
        "Line Pressure (PSIA).csv"
    ]
    for col in cols:
        cycle_df[col] = cycle_df[col].astype(float)

    Pt_init = cycle_df.iloc[0]["Tubing Pressure (PSI).csv"]
    Pt_final = cycle_df.iloc[-1]["Tubing Pressure (PSI).csv"]
    Cp_init = cycle_df.iloc[0]["Casing Pressure (PSI).csv"]
    Cp_final = cycle_df.iloc[-1]["Casing Pressure (PSI).csv"]
    Pl_init = cycle_df.iloc[0]["Line Pressure (PSIA).csv"]
    Pl_final = cycle_df.iloc[-1]["Line Pressure (PSIA).csv"]

    delta_Pt = Pt_final - Pt_init
    delta_Cp = Cp_final - Cp_init
    delta_Pl = Pl_final - Pl_init
    ph = 0.433 * SG * hl

    return {"BasicPressureEvent": {
        "cycle_id": int(cycle_df.iloc[0]["cycle_id"]),
        "delta_Pt": round(float(delta_Pt), 3),
        "delta_Cp": round(float(delta_Cp), 3),
        "delta_Pl": round(float(delta_Pl), 3),
        "ph": round(float(ph), 3)
    }}

In [101]:
def CycleDurationEvent(cycle_df):
    # Start and end time of the cycle
    start_time = int(cycle_df.iloc[0]['isotime'])
    end_time = int(cycle_df.iloc[-1]['isotime'])
    total_duration = end_time - start_time

    # Flow duration: time when flow_rate was not zero
    non_zero = cycle_df[cycle_df['flow_rate'] > 0]
    if not non_zero.empty:
        flow_start = int(non_zero.iloc[0]['isotime'])
        flow_end = int(non_zero.iloc[-1]['isotime'])
        flow_duration = flow_end - flow_start
    else:
        flow_duration = 0

    # Shutin duration: time when flow_rate was zero
    zero = cycle_df[cycle_df['flow_rate'] == 0]
    if not zero.empty:
        shutin_start = int(zero.iloc[0]['isotime'])
        shutin_end = int(zero.iloc[-1]['isotime'])
        shutin_duration = shutin_end - shutin_start
    else:
        shutin_duration = 0

    return {"CycleDurationEvent": {
        "cycle_id": int(cycle_df.iloc[0]['cycle_id']),
        "start_time": start_time,
        "end_time": end_time,
        "total_duration": total_duration,
        "flow_duration": flow_duration,
        "shutin_duration": shutin_duration
    }}

In [102]:
def PlungerArrivalVelocityEvent(cycle_df):
    # Convert Arrival Speed.csv to float and get mean arrival speed for the cycle
    arrival_speed = cycle_df["Arrival Speed.csv"].astype(float).mean()
    return {
        "PlungerArrivalVelocityEvent": {
            "cycle_id": int(cycle_df.iloc[0]["cycle_id"]),
            "arrival_speed": round(float(arrival_speed), 3),  # m/s
        }
    }

complex:

In [103]:
def GasVolumeProducedEvent(cycle_df, events):
    # Find the CycleDurationEvent in the provided events
    cycle_duration_event = next((e["CycleDurationEvent"] for e in events if "CycleDurationEvent" in e), None)
    if cycle_duration_event is None:
        return {"GasVolumeProducedEvent": None}

    flow_duration = cycle_duration_event["flow_duration"]  # in seconds
    # Use mean flow_rate during the cycle (excluding zeros)
    non_zero_flow = cycle_df[cycle_df['flow_rate'] > 0]
    if not non_zero_flow.empty:
        avg_flow_rate = non_zero_flow['flow_rate'].mean()  # MCF/Day
        # Convert avg_flow_rate from MCF/Day to cubic meters/second
        # 1 MCF = 28.3168 m³, 1 day = 86400 seconds
        avg_flow_rate_m3s = avg_flow_rate * 28.3168 / 86400
        gas_volume = avg_flow_rate_m3s * flow_duration  # in cubic meters
    else:
        gas_volume = 0.0

    return {
        "GasVolumeProducedEvent": {
            "cycle_id": int(cycle_df.iloc[0]["cycle_id"]),
            "gas_volume": round(float(gas_volume), 3),  # cubic meters
            "CycleDurationEvent": cycle_duration_event  # in seconds
        }
    }

In [104]:
def CycleDataEvent(cycle_df, events):
    event_log = {
        "cycle_id": int(cycle_df.iloc[0]["cycle_id"])
    }
    # Map event names to their data
    for event in events:
        if "BasicPressureEvent" in event:
            event_log["basicPressureEvent"] = event["BasicPressureEvent"]
        elif "CycleDurationEvent" in event:
            event_log["cycleDurationEvent"] = event["CycleDurationEvent"]
        elif "GasVolumeProducedEvent" in event:
            event_log["gasVolumeProduced"] = event["GasVolumeProducedEvent"]
        elif "PlungerArrivalVelocityEvent" in event:
            event_log["velocityEvent"] = event["PlungerArrivalVelocityEvent"]
    return {"CycleDataEvent": event_log}

In [105]:
def UnexpectedLowCasingPressure(cycle_df, events, threshold=-5.0):
    basic_pressure_event = next((e["BasicPressureEvent"] for e in events if "BasicPressureEvent" in e))

    delta_cp = basic_pressure_event['delta_Cp']
    if delta_cp < threshold:
        return {
            "UnexpectedLowCasingPressure": {
                "cycle_id": int(cycle_df.iloc[0]["cycle_id"]),
                "BasicPressureEvent": basic_pressure_event,
                "description": "Abnormally low casing pressure change detected."
            }
        }
    return None

In [106]:
def PlungerArrivalStatusEvent(cycle_df, events):
    # non_arrival: True if all Current Non-Arrival Count.csv > 0, else False
    non_arrival = cycle_df["Current Non-Arrival Count.csv"].astype(float).max() > 0

    unexpected_casing_pressure = next((e["UnexpectedLowCasingPressure"] for e in events if "UnexpectedLowCasingPressure" in e), None)

    return {
        "PlungerArrivalStatusEvent": {
            "cycle_id": int(cycle_df.iloc[0]["cycle_id"]),
            "non_arrival": bool(non_arrival),
            "unexpected_casing_pressure": bool(unexpected_casing_pressure),
            "UnexpectedLowCasingPressure": unexpected_casing_pressure,
            "description": (
                "Plunger did not arrive; " if non_arrival else "Plunger arrived; "
            ) + (
                "Unexpected low casing pressure detected."
                if unexpected_casing_pressure
                else "Casing pressure normal."
            ),
        }
    }

In [107]:
def PlungerUnsafeVelocityEvent(cycle_df, events, safety_threshold=2.5):
    # Find the PlungerArrivalVelocityEvent in the provided events
    velocity_event = next((e["PlungerArrivalVelocityEvent"] for e in events if "PlungerArrivalVelocityEvent" in e), None)
    if velocity_event is None:
        return None

    arrival_speed = velocity_event["arrival_speed"]
    unsafe = arrival_speed > safety_threshold

    if unsafe:
        return {
            "PlungerUnsafeVelocity": {
                "cycle_id": int(cycle_df.iloc[0]["cycle_id"]),
                "arrival_speed": arrival_speed,
                "velocityEvent": velocity_event,
                "description": (
                    "The PlungerUnsafeVelocity event is triggered when the velocity of the plunger upon arrival "
                    "at the surface exceeds a predefined safety threshold. This indicates potentially dangerous "
                    "impact forces that could damage equipment or signal aggressive flow conditions. "
                    "Operators should consider adjusting flow duration, shut-in pressure, or inspect for mechanical wear."
                ),
                "safety_threshold": safety_threshold
            }
        }
    return None

In [108]:
def UnexpectedLowFlow(cycle_df, events, volume_threshold=10.0):
    gas_volume_event = next((e["GasVolumeProducedEvent"] for e in events if "GasVolumeProducedEvent" in e), None)
    if gas_volume_event is None:
        return None

    gas_volume = gas_volume_event["gas_volume"]
    if gas_volume < volume_threshold:
        return {
            "UnexpectedLowFlow": {
                "cycle_id": int(cycle_df.iloc[0]["cycle_id"]),
                "gas_volume": gas_volume,
                "volume_threshold": volume_threshold,
                "description": (
                    "The UnexpectedLowFlow event is raised when the gas volume produced during a cycle is significantly lower than expected. "
                    "This may indicate underperformance, early plunger fallback, or poor liquid unloading."
                ),
            }
        }
    return None

In [112]:
def UnexpectedLowCycleDuration(cycle_df, events, total_duration_threshold=600, flow_duration_threshold=300, shutin_duration_threshold=300):
    # Find the CycleDurationEvent in the provided events
    cycle_duration_event = next((e["CycleDurationEvent"] for e in events if "CycleDurationEvent" in e), None)
    if cycle_duration_event is None:
        return None

    total_duration = cycle_duration_event["total_duration"]
    flow_duration = cycle_duration_event["flow_duration"]
    shutin_duration = cycle_duration_event["shutin_duration"]

    is_short = (
        total_duration < total_duration_threshold or
        flow_duration < flow_duration_threshold or
        shutin_duration < shutin_duration_threshold
    )

    if is_short:
        return {
            "UnexpectedLowCycleDuration": {
                "cycle_id": int(cycle_df.iloc[0]["cycle_id"]),
                "total_duration": total_duration,
                "flow_duration": flow_duration,
                "shutin_duration": shutin_duration,
                "thresholds": {
                    "total_duration": total_duration_threshold,
                    "flow_duration": flow_duration_threshold,
                    "shutin_duration": shutin_duration_threshold
                },
                "CycleDurationEvent": cycle_duration_event,
                "description": (
                    "The UnexpectedLowCycleDuration event flags abnormally short cycles, either in total duration or during specific segments like flow or shut-in. "
                    "Such a condition could suggest premature venting, shallow slug formation, or mistimed plunger launches. "
                    "Identifying low-duration anomalies is important to optimize cycle timing and avoid inefficient runtimes."
                )
            }
        }
    return None

In [109]:
def UnexpectedHighCycleDuration(cycle_df, events, total_duration_threshold=7200, flow_duration_threshold=3600, shutin_duration_threshold=3600):
    cycle_duration_event = next((e["CycleDurationEvent"] for e in events if "CycleDurationEvent" in e), None)
    if cycle_duration_event is None:
        return None

    total_duration = cycle_duration_event["total_duration"]
    flow_duration = cycle_duration_event["flow_duration"]
    shutin_duration = cycle_duration_event["shutin_duration"]

    is_long = (
        total_duration > total_duration_threshold or
        flow_duration > flow_duration_threshold or
        shutin_duration > shutin_duration_threshold
    )

    if is_long:
        return {
            "UnexpectedHighCycleDuration": {
                "cycle_id": int(cycle_df.iloc[0]["cycle_id"]),
                "total_duration": total_duration,
                "flow_duration": flow_duration,
                "shutin_duration": shutin_duration,
                "thresholds": {
                    "total_duration": total_duration_threshold,
                    "flow_duration": flow_duration_threshold,
                    "shutin_duration": shutin_duration_threshold
                },
                "CycleDurationEvent": cycle_duration_event,
                "description": (
                    "The UnexpectedHighCycleDuration event highlights cycles that exceed acceptable time limits, "
                    "particularly during flow or shut-in phases. This can indicate poor liquid unloading, sluggish arrival, "
                    "excessive shut-in, or gas buildup delays. By identifying these long cycles, operators can rebalance lift "
                    "frequency, flow durations, and shut-in strategies to restore cycle efficiency."
                )
            }
        }
    return None

In [110]:
def CycleAnomalyEvent(cycle_df, events):
    # Map anomaly event keys to their output names and descriptions
    anomaly_map = {
        "PlungerUnsafeVelocity": "plungerUnsafeVelocity",
        "PlungerNonArrival": "plungerNonArrival",
        "UnexpectedLowCasingPressure": "UnexpectedCasingPressure",
        "UnexpectedLowFlow": "UnexpectedLowFlow",
        "UnexpectedLowCycleDuration": "UnexpectedLowCycleDuration",
        "UnexpectedHighCycleDuration": "UnexpectedHighCycleDuration"
    }

    # Find which anomaly events are present in the events list
    triggered = {}
    for event in events:
        for key, output_name in anomaly_map.items():
            if key in event:
                triggered[output_name] = event[key]

    if triggered:
        return {
            "CycleAnomalyEvent": {
                "cycle_id": int(cycle_df.iloc[0]["cycle_id"]),
                "anomalies": list(triggered.keys()),
                "details": triggered,
                "description": (
                    "The CycleAnomalyEvent is a composite event designed to encapsulate critical failures, "
                    "inefficiencies, or safety violations occurring during a plunger lift cycle. It is triggered "
                    "whenever one or more constituent anomaly events occur, including PlungerNonArrival, "
                    "PlungerUnsafeVelocity, UnexpectedLowCasingPressure, UnexpectedLowFlow, "
                    "UnexpectedLowCycleDuration, and UnexpectedHighCycleDuration. This wrapper event provides "
                    "a high-level signal indicating that a cycle has deviated from expected operational behavior. "
                    "It supports automated monitoring and root-cause analysis by summarizing which specific "
                    "anomalies occurred, helping identify equipment issues, cycle misconfigurations, or poor lift "
                    "conditions that warrant intervention."
                )
            }
        }
    return None

In [113]:
generate_events_per_cycle(
    df, 
    [BasicPressureEvent, CycleDurationEvent, PlungerArrivalVelocityEvent],
    [GasVolumeProducedEvent, CycleDataEvent, UnexpectedLowCasingPressure, PlungerArrivalStatusEvent, PlungerUnsafeVelocityEvent, UnexpectedLowCycleDuration, UnexpectedLowFlow, CycleAnomalyEvent]
)

[{'BasicPressureEvent': {'cycle_id': 0,
   'delta_Pt': -83.08,
   'delta_Cp': -0.3,
   'delta_Pl': 0.0,
   'ph': 259.8}},
 {'CycleDurationEvent': {'cycle_id': 0,
   'start_time': 1751184740,
   'end_time': 1751185820,
   'total_duration': 1080,
   'flow_duration': 720,
   'shutin_duration': 240}},
 {'PlungerArrivalVelocityEvent': {'cycle_id': 0, 'arrival_speed': 1021.666}},
 {'GasVolumeProducedEvent': {'cycle_id': 0,
   'gas_volume': 153.868,
   'CycleDurationEvent': {'cycle_id': 0,
    'start_time': 1751184740,
    'end_time': 1751185820,
    'total_duration': 1080,
    'flow_duration': 720,
    'shutin_duration': 240}}},
 {'CycleDataEvent': {'cycle_id': 0,
   'basicPressureEvent': {'cycle_id': 0,
    'delta_Pt': -83.08,
    'delta_Cp': -0.3,
    'delta_Pl': 0.0,
    'ph': 259.8},
   'cycleDurationEvent': {'cycle_id': 0,
    'start_time': 1751184740,
    'end_time': 1751185820,
    'total_duration': 1080,
    'flow_duration': 720,
    'shutin_duration': 240},
   'velocityEvent': {'cyc